# BNSynthesis-generator

## Instructions

1. Set `N` and `M` to the desired values

1. Run the script
   - Just run the cell
   - This will generate a python script with fully underspecified original BN `(X,F)`, abstraction function `G`, and abstract BN `(Y,H)`
   - The script will be:
     - Added to a new cell if running on Jupiter-lab
     - Printed in output if running on Google colab

1. Refine any of the above at will (look for comment "# Replace rhs with actual constraints")
   - For example, to synthetize an abstraction function for a given original BN, one can specify the original BN by specifying each update function `fi` like this
> `ForAll([x0,x1,..,xN],fi(x0,x1,..,xN) == ...`
   -The LHS is ready in the generated file. One has just to replace the RHS.

1. Run the new script as usual
   - This will generate possible ways of completing `F`, `G` and `H`


# Script generator

In [8]:

import datetime
now = datetime.datetime.now()

# Size of original BN
N = 3
# Size of abstracted BN
M = 2

output = "from IPython.display import display, Markdown#, Latex\n"
output += "display(Markdown('__Generated on "+str(now)+"__'))\n"
output += 'display(Markdown("N='+str(N)+', M='+str(M)+'"))\n'

output += 'import sys\n'
output += "IN_COLAB = 'google.colab' in sys.modules\n"
output += 'if IN_COLAB:\n'
output += '    print("Running on Google colab")\n'
output += 'else:\n'
output += '    print("Running on Jupyter lab")\n'

output += "\n\n"

output += "# Install z3 for python as explained here\n"
output += "# https://github.com/Z3Prover/z3\n"
output += "# Or just run the following command...\n"
output += "# %pip install matplotlib\n"
output += "from z3 import *\n"
output += "\n"

output += "# The solver\n"
output += "solver = Solver()\n"
output += "\n"

output += "# Original BN update function F = f0..fn-1\n"
for i in range(N):
    output += "f" + str(i) + " = Function('f" + str(i) + "', "
    for j in range(N):
        output += "BoolSort(),"
    output += "BoolSort())\n"
output += "\n"

output += "# Some Boolean variables for original BN\n"
for i in range(N):
    output += "x" + str(i) + " = Const('x" + str(i) + "' , BoolSort())"
    output += "\n"
output += "\n"

output += "# Abstraction function G = g0..gn-1\n"
for i in range(M):
    output += "g" + str(i) + " = Function('g" + str(i) + "', "
    for j in range(N):
        output += "BoolSort(),"
    output += "BoolSort())\n"
output += "\n"

output += "# Abstract BN update function H = h1..hn-1\n"
for i in range(M):
    output += "h" + str(i) + " = Function('h" + str(i) + "', "
    for j in range(M):
        output += "BoolSort(),"
    output += "BoolSort())\n"
output += "\n"

output += "# Some Boolean variables\n"
for i in range(M):
    output += "y" + str(i) + " = Const('y" + str(i) + "' , BoolSort())"
    output += "\n"
output += "\n"

output += "# Constraints (partially) defining F, G and H\n"
output += "solver.add(\n"
output+="    # Vacuous constraints for f to serve as placehoders\n"
output+="    # Replace rhs with actual constraints\n"
for i in range(N):
    output += "    ForAll(["
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+="],f" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+=") == f" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+=")),\n"
output+="\n"
output+="    # Vacuous constraints for g to serve as placehoders\n"
output+="    # Replace rhs with actual constraints\n"
for i in range(M):
    output += "    ForAll(["
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+="],g" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+=") == g" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+=")),\n"
output+="\n"
output+="    # Vacuous constraints for h to serve as placehoders\n"
output+="    # Replace rhs with actual constraints\n"
for i in range(M):
    output += "    ForAll(["
    for j in range(M):
        output+="x" + str(j)
        if j<(M-1): output+=","
    output+="],h" + str(i) + "("
    for j in range(M):
        output+="x" + str(j)
        if j<(M-1): output+=","
    output+=") == h" + str(i) + "("
    for j in range(M):
        output+="x" + str(j)
        if j<(M-1): output+=","
    output+=")),\n"
output+="\n"    
output+="    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)\n"
output += "    ForAll(["
for i in range(N):
        output+="x" + str(i)
        if i<(N-1): output+=","
output += "], And(\n"
for k in range(M):
    output += "      g" + str(k) + "("
    for l in range(N):
        output += "f" + str(l) + "("
        for j in range(N):
            output+="x" + str(j)
            if j<(N-1): output+=","
            else: output+=")"
        if l<(N-1): output+=","
    output+=")"
    output += " == h" + str(k) + "("
    for l in range(M):
        output += "g" + str(l) + "("
        for j in range(N):
            output+="x" + str(j)
            if j<(N-1): output+=","
            else: output+=")"
        if l<(M-1): output+=","
    output += ")"
    if k<(M-1): output += ",\n"
output += "\n    ))\n"
output += ")\n\n"

output += "#While loop providing solutions\n"
output += "while solver.check() == sat:\n"
output += "    m = solver.model()\n"
output += "    print(\"Hoorray! Here is a possible solution:\")\n"
output += "    print(m)\n"
output += "    print(\"Press [enter] for a different one, or type stop to terminate...\")\n"
output += "    string = input()\n"
output += "    if string == 'STOP' or string == 'stop':\n"
output += "        print('I terminate, as required')\n"
output += "        break\n"
output += "    solver.add(Or([\n"
output += "      # Constraints to obtain a different solution (for now only a different G)\n"
#output += "    [\n"
for i in range(M):
    output += "      Not(ForAll(["
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+="],m.eval(g" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+=")) == g" + str(i) + "("
    for j in range(N):
        output+="x" + str(j)
        if j<(N-1): output+=","
    output+="))),\n"

output += "      False]\n"
output += "    ))\n"
output += "else:\n"
output += "    print(\"No solution, sorry!\")\n"

print("Generation completed.")
#print(output)

#from IPython.display import display, Markdown#, Latex
#display(Markdown("N="+str(N)+", M="+str(M)))
#display(Markdown("- 1"))
#display(Markdown("- 2"))
#get_ipython().set_

if IN_COLAB:
    print(output)
else:
    print("Creating new cell with computed output...")
    get_ipython().set_next_input(output,replace=False)
    print("  done")

Generation completed.
Creating new cell with computed output...
  done


# Precomputed scripts

## Original generated script

In [ ]:
#N = 3, M = 2

# Install z3 for python as explained here
# https://github.com/Z3Prover/z3
from z3 import *

# The solver
solver = Solver()

# Original BN update function F = f0..fn-1
f0 = Function('f0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f1 = Function('f1', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f2 = Function('f2', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables for original BN
x0 = Const('x0' , BoolSort())
x1 = Const('x1' , BoolSort())
x2 = Const('x2' , BoolSort())

# Abstraction function G = g0..gn-1
g0 = Function('g0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
g1 = Function('g1', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Abstract BN update function H = h1..hn-1
h0 = Function('h0', BoolSort(),BoolSort(),BoolSort())
h1 = Function('h1', BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables
y0 = Const('y0' , BoolSort())
y1 = Const('y1' , BoolSort())

# Constraints (partially) defining F, G and H
solver.add(
    # Vacuous constraints for f to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],f0(x0,x1,x2) == f0(x0,x1,x2)),
    ForAll([x0,x1,x2],f1(x0,x1,x2) == f1(x0,x1,x2)),
    ForAll([x0,x1,x2],f2(x0,x1,x2) == f2(x0,x1,x2)),

    # Vacuous constraints for g to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],g0(x0,x1,x2) == g0(x0,x1,x2)),
    ForAll([x0,x1,x2],g1(x0,x1,x2) == g1(x0,x1,x2)),

    # Vacuous constraints for h to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1],h0(x0,x1) == h0(x0,x1)),
    ForAll([x0,x1],h1(x0,x1) == h1(x0,x1)),

    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)
    ForAll([x0,x1,x2], And(
      g0(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h0(g0(x0,x1,x2),g1(x0,x1,x2)),
      g1(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h1(g0(x0,x1,x2),g1(x0,x1,x2))
    ))
)

#While loop providing solutions
while solver.check() == sat:
    m = solver.model()
    print("Hoorray! Here is a possible solution:")
    print(m)
    print("Press [enter] for a different one, or type stop to terminate...")
    string = input()
    if string == 'STOP' or string == 'stop':
        print('I terminate, as required')
        break
    solver.add(Or([
      # Constraints to obtain a different solution (for now only a different G)
      Not(ForAll([x0,x1,x2],m.eval(g0(x0,x1,x2)) == g0(x0,x1,x2))),
      Not(ForAll([x0,x1,x2],m.eval(g1(x0,x1,x2)) == g1(x0,x1,x2))),
      False]))
else:
    print("No solution, sorry!")

Hoorray! Here is a possible solution:
[g0 = [else -> False],
 f0 = [else -> False],
 f2 = [else -> False],
 f1 = [else -> False],
 h0 = [else -> False],
 h1 = [else -> False],
 g1 = [else -> False]]
Press [enter] for a different one, or type stop to terminate...


 stop


I terminate, as required


## Refined generated script

In [ ]:
#N = 3, M = 2

# Install z3 for python as explained here
# https://github.com/Z3Prover/z3
from z3 import *

# The solver
solver = Solver()

# Original BN update function F = f0..fn-1
f0 = Function('f0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f1 = Function('f1', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f2 = Function('f2', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables for original BN
x0 = Const('x0' , BoolSort())
x1 = Const('x1' , BoolSort())
x2 = Const('x2' , BoolSort())

# Abstraction function G = g0..gn-1
g0 = Function('g0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
g1 = Function('g1', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Abstract BN update function H = h1..hn-1
h0 = Function('h0', BoolSort(),BoolSort(),BoolSort())
h1 = Function('h1', BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables
y0 = Const('y0' , BoolSort())
y1 = Const('y1' , BoolSort())

# Constraints (partially) defining F, G and H
solver.add(
    # Vacuous constraints for f to serve as placehoders
    # Replace rhs with actual constraints
    #ForAll([x0,x1,x2],f0(x0,x1,x2) == f0(x0,x1,x2)),
    #ForAll([x0,x1,x2],f1(x0,x1,x2) == f1(x0,x1,x2)),
    #ForAll([x0,x1,x2],f2(x0,x1,x2) == f2(x0,x1,x2)),
    # BN from Georgios slides
    ForAll([x0,x1,x2],f0(x0,x1,x2) == Not(x1)) ,
    ForAll([x0,x1,x2],f1(x0,x1,x2) == Or(Not(x0),Not(x2))) ,
    ForAll([x0,x1,x2],f2(x0,x1,x2) == Or([Not(x0),x1,x2])) ,

    # Vacuous constraints for g to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],g0(x0,x1,x2) == g0(x0,x1,x2)),
    ForAll([x0,x1,x2],g1(x0,x1,x2) == g1(x0,x1,x2)),

    # Vacuous constraints for h to serve as placehoders
    # Replace rhs with actual constraints
    #ForAll([x0,x1],h0(x0,x1) == h0(x0,x1)),
    #ForAll([x0,x1],h1(x0,x1) == h1(x0,x1)),
    # Aggregation function slide 2 - Does not provide a bisimilar deterministic/synchronous BN
    #ForAll([x0,x1,x2],g0(x0,x1,x2) == Or(x0,x1)) ,
    #ForAll([x0,x1,x2],g1(x0,x1,x2) == x2) ,
    # Aggregation function slide 6 - Provides a unique bisimilar deterministic/synchronous BN
    ForAll([x0,x1,x2],g0(x0,x1,x2) == Or(Not(x0),x1)) ,
    ForAll([x0,x1,x2],g1(x0,x1,x2) == x2) ,

    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)
    ForAll([x0,x1,x2], And(
      g0(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h0(g0(x0,x1,x2),g1(x0,x1,x2)),
      g1(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h1(g0(x0,x1,x2),g1(x0,x1,x2))
    ))
)

#While loop providing solutions
while solver.check() == sat:
    m = solver.model()
    print("Hoorray! Here is a possible solution:")
    print(m)
    print("Press [enter] for a different one, or type stop to terminate...")
    string = input()
    if string == 'STOP' or string == 'stop':
        print('I terminate, as required')
        break
    solver.add(Or([
      # Constraints to obtain a different solution (for now only a different G)
      Not(ForAll([x0,x1,x2],m.eval(g0(x0,x1,x2)) == g0(x0,x1,x2))),
      Not(ForAll([x0,x1,x2],m.eval(g1(x0,x1,x2)) == g1(x0,x1,x2))),
      False]))
else:
    print("No solution, sorry!")

Hoorray! Here is a possible solution:
[f0 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))))],
 g0 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Var(1), Var(2)),
          And(Var(0), Var(1), Not(Var(2))),
          And(Not(Var(0)), Var(1), Not(Var(2))),
          And(Var(0), Var(1), Var(2)))],
 h1 = [(False, False) -> False, else -> True],
 f2 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Var(2)),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Var(1), Var(2)),
          And(Var(0), Var(1), Not(Var(2))),
          And(Not(Var(0)), Var(1), Not(Var(2))),
          And(Var(0), Var(1), Var(2)))],
 h0 = [(False, True) -> False, else -> True],
 f1 = [else ->
     

 stop


I terminate, as required
